In [14]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Flatten, Input
from tensorflow.keras.models import Model
from qkeras import QDense, QActivation, quantized_bits

In [15]:
# Definicja modelu, wzór
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(200, activation='sigmoid'),
    Dense(80, activation='sigmoid'),
    Dense(10, activation='sigmoid')
])

In [16]:
from tensorflow.keras.datasets import mnist

# Pobieranie danych MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# Sprawdzenie kształtów danych
print("Kształt X_train:", X_train.shape)
print("Kształt y_train:", y_train.shape)
print("Kształt X_test:", X_test.shape)
print("Kształt y_test:", y_test.shape)


Kształt X_train: (60000, 28, 28)
Kształt y_train: (60000,)
Kształt X_test: (10000, 28, 28)
Kształt y_test: (10000,)


In [17]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(200, activation='sigmoid'),
    Dense(80, activation='sigmoid'),
    Dense(10, activation='sigmoid')
])


In [18]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test))

Epoch 1/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5206 - accuracy: 0.8620 - val_loss: 0.3257 - val_accuracy: 0.9033
Epoch 2/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3275 - accuracy: 0.9016 - val_loss: 0.2967 - val_accuracy: 0.9114
Epoch 3/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2917 - accuracy: 0.9103 - val_loss: 0.2693 - val_accuracy: 0.9145


Analizujemy kolejne stopnie kwantyzacji, każdy po 5 razy żeby można było policzyć statystykę. Za każdym razem na nowo losujemy podział train_test_split

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
acc = np.zeros((8, 5))
for j in range(0, 5):
    #robimy tak bo zbiór z tensorflow jest domyślnie podzielony na test i train, a my chcemy to podzielić losowo
    X_full = np.concatenate((X_train, X_test))
    y_full = np.concatenate((y_train, y_test))

    X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2)
    for i in range (1, 9):
        q_bits = i
        input_shape = Input(shape=(28, 28))
        x = Flatten()(input_shape)
        x = QDense(200, kernel_quantizer=quantized_bits(i, 0, 1), bias_quantizer=quantized_bits(i, 0, 1))(x)
        x = QActivation("quantized_sigmoid({})".format(i))(x)
        x = QDense(80, kernel_quantizer=quantized_bits(i, 0, 1), bias_quantizer=quantized_bits(i, 0, 1))(x)
        x = QActivation("quantized_sigmoid({})".format(i))(x)
        x = QDense(10, kernel_quantizer=quantized_bits(i, 0, 1), bias_quantizer=quantized_bits(i, 0, 1))(x)
        x = QActivation("quantized_sigmoid({}".format(i))(x)
        q_model = Model(inputs=input_shape, outputs=x)

        q_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        q_model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test))
        acc[i-1, j] = q_model.evaluate(X_test, y_test, verbose=0)[1]
        print(acc[i-1, j])

TypeError: Exception encountered when calling layer "q_activation_3" (type QActivation).

in user code:

    File "/home/bstepniewski/.local/lib/python3.10/site-packages/qkeras/qlayers.py", line 177, in call  *
        return self.quantizer(inputs)
    File "/home/bstepniewski/.local/lib/python3.10/site-packages/qkeras/quantizers.py", line 1774, in __call__  *
        m = K.cast_to_floatx(K.pow(2, self.bits))
    File "/home/bstepniewski/.local/lib/python3.10/site-packages/keras/src/backend.py", line 3140, in pow
        return tf.pow(x, a)

    TypeError: Expected int32 passed to parameter 'y' of op 'Pow', got 'i' of type 'str' instead. Error: Expected int32, but got i of type 'str'.


Call arguments received by layer "q_activation_3" (type QActivation):
  • inputs=tf.Tensor(shape=(None, 200), dtype=float32)